NLP Embedding Sentence Transformer

In [ ]:
from sentence_transformers import SentenceTransformer, util
from torch.quantization import quantize_dynamic
model_st = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
model_qt = quantize_dynamic(model_st)
#embeddings = model_qt.encode(utterance_text_ls)
#embeddings.shape

Add Embeddings to dataframe

In [ ]:
embedding_columns = [f'Embedding_{i}' for i in range(hashtag_embedding.shape[1])]
df[embedding_columns] = hashtag_embedding

Fasttext

In [ ]:
#from huggingface_hub import hf_hub_download

model_path = hf_hub_download(repo_id="facebook/fasttext-en-vectors", filename="model.bin")
model = fasttext.load_model(model_path)
model.predict("Hello, world!")

In [ ]:
model = fasttext.load_model(model_path)
model = fasttext.train_supervised(input="formatted_data.txt", maxn = 9)
model.save_model("model_ft.bin")

In [ ]:
# Define the output file path
output_file = 'formatted_data.txt'

# Open the file for writing
with open(output_file, 'w') as file:
    # Iterate through DataFrame rows
    for index, row in df.iterrows():
        # Format Column1 and Column2 as "__label__word1-word2"
        formatted_column1 = "__label__" + row["reorg_issue"].replace(" ", "-")
        formatted_column2 = "__label__" + row["reorg_sub_issue"].replace(" ", "-")

        # Column3 is concatenated as is
        formatted_column3 = row["challenge_name"]

        # Combine all formatted columns and write to the file
        formatted_row = f'{formatted_column1} {formatted_column2} {formatted_column3}\n'
        file.write(formatted_row)

print(f"Formatted data saved to {output_file}")

In [ ]:
output_file = 'formatted_data.txt'
with open(output_file, 'r') as file:
    text = file.read()

Ordinal Encoding for categorical variable

In [ ]:
# Handling the categorical data
vv_mapping = {k: i for i, k in enumerate(
   ["(0, 1K]","(1K, 10K]","(10K, 100K]","(100K, 1M]","(1M, 10M]","10M+"], 0)}
df.video_vv_strata = df.video_vv_strata.map(vv_mapping)
publish_mapping = {k: i for i, k in enumerate(
   ['(0, 20]','(20, 100]','(100, 1K]','(1K, 10K]','(10K, 100K]','100K+'], 0)}
df.video_publish_strata = df.video_publish_strata.map(publish_mapping)
df.top1_video_publish_coutnry_code_15d = df.top1_video_publish_coutnry_code_15d.astype('category')

One hot encoding

In [ ]:
  # Define the column to be one-hot encoded\n",
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
ohe = OneHotEncoder(sparse=False)
categorical_column = ['top1_video_publish_coutnry_code_15d']  # Name of the column to be encoded\n",

    # Define a function to extract the feature names\n",
def get_feature_names(ohe, input_features):
        return ohe.get_feature_names_out(input_features)

    # Initialize the ColumnTransformer\n",
transformer = ColumnTransformer(
       transformers=[
      ('onehot', OneHotEncoder(), categorical_column)],remainder='passthrough')  # Pass through other columns unchanged\n",

    # Fit and transform the data\n",
encoded_data = transformer.fit_transform(hashtag_clustering)
    # Get the transformed feature names\n",
feature_names = get_feature_names(transformer.named_transformers_['onehot'], categorical_column)
    # Create a DataFrame with the transformed data and feature names",
#client = pd.DataFrame(encoded_data, columns=list(feature_names) + list(client.columns[1:]))",

Standardizing data

In [ ]:
from sklearn.preprocessing import StandardScaler

# Create a StandardScaler object
scaler = StandardScaler()

# Fit and transform the data
#scaled_data = scaler.fit_transform(df.iloc[:,list(range(9,16))])
scaled_data = scaler.fit_transform(hashtag_clustering.iloc[:,1:-1])
hashtag_clustering.iloc[:,1:-1] = scaled_data

Histogram

In [ ]:
import matplotlib.pyplot as plt

# Create a histogram
plt.hist(violation_rate['violation_rate_15d'], bins=5, color='skyblue', edgecolor='black')  # Adjust the number of bins as needed

# Add labels and title
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('Histogram Plot')

# Show the plot
plt.show()

K means clustering

In [ ]:
from sklearn.cluster import KMeans

# Create a K-Means model with the desired number of clusters (k)
kmeans = KMeans(n_clusters=2)

# Fit the model to your numerical data
kmeans.fit(scaled_data)

# Get cluster assignments for each data point
cluster_labels = kmeans.labels_

Use silouette plot and silouette score to determine what is the best number of cluster, the higher the better

In [ ]:
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import numpy as np

from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from sklearn.metrics import silhouette_samples, silhouette_score
X = scaled_data


range_n_clusters = [2, 3, 4, 5, 6]

for n_clusters in range_n_clusters:
    # Create a subplot with 1 row and 2 columns
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(18, 7)

    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1 but in this example all
    # lie within [-0.1, 1]
    ax1.set_xlim([-0.1, 1])
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])

    # Initialize the clusterer with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    clusterer = KMeans(n_clusters=n_clusters)
    cluster_labels = clusterer.fit_predict(X)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(X, cluster_labels)
    print(
        "For n_clusters =",
        n_clusters,
        "The average silhouette_score is :",
        silhouette_avg,
    )

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(X, cluster_labels)

    y_lower = 10
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(
            np.arange(y_lower, y_upper),
            0,
            ith_cluster_silhouette_values,
            facecolor=color,
            edgecolor=color,
            alpha=0.7,
        )

        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title("The silhouette plot for the various clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

    # 2nd Plot showing the actual clusters formed
    colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
    ax2.scatter(
        X[:, 0], X[:, 1], marker=".", s=30, lw=0, alpha=0.7, c=colors, edgecolor="k"
    )

    # Labeling the clusters
    centers = clusterer.cluster_centers_
    # Draw white circles at cluster centers
    ax2.scatter(
        centers[:, 0],
        centers[:, 1],
        marker="o",
        c="white",
        alpha=1,
        s=200,
        edgecolor="k",
    )

    for i, c in enumerate(centers):
        ax2.scatter(c[0], c[1], marker="$%d$" % i, alpha=1, s=50, edgecolor="k")

    ax2.set_title("The visualization of the clustered data.")
    ax2.set_xlabel("Feature space for the 1st feature")
    ax2.set_ylabel("Feature space for the 2nd feature")

    plt.suptitle(
        "Silhouette analysis for KMeans clustering on sample data with n_clusters = %d"
        % n_clusters,
        fontsize=14,
        fontweight="bold",
    )

plt.show()

Hierachical clustering

For dendrogram, have a horizontal line as the threshold that cuts off, each intersection is a cluster. The y axis is the distance apart in between each cluster.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage


# Perform hierarchical clustering
linkage_matrix = linkage(scaled_data, method='ward')  # You can choose different linkage methods
# Plot the dendrogram
plt.figure(figsize=(10, 6))
dendrogram(linkage_matrix, leaf_rotation=90, leaf_font_size=12)
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('Data Points')
plt.ylabel('Distance')
plt.show()

# Choose a threshold or number of clusters (e.g., by visually inspecting the dendrogram)
threshold = 50  # Adjust this value based on your dendrogram

# Extract clusters based on the threshold
clusters = fcluster(linkage_matrix, t=threshold, criterion='distance')

# Assign cluster labels to data points
your_data['Cluster'] = clusters


Isolation Forest

In [ ]:
import numpy as np
from sklearn.ensemble import IsolationForest
# Visualize the results (optional)
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA


# Create and fit the Isolation Forest model
model = IsolationForest(contamination=0.1)  # Contamination parameter defines the expected proportion of anomalies
model.fit(hashtag_clustering.iloc[:,0:-1])

# Predict outliers (1 for inliers, -1 for outliers)
y_pred = model.predict(hashtag_clustering.iloc[:,0:-1])

# Apply PCA to reduce the data to 2D for visualization
pca = PCA(n_components=2)
X_reduced = pca.fit_transform(hashtag_clustering.iloc[:,0:-1])

# Visualize the results
plt.scatter(X_reduced[:, 0], X_reduced[:, 1], c=y_pred, cmap='viridis')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('Isolation Forest Anomaly Detection (PCA Visualization)')
plt.show()

Supervised Learning

Oversampling on the imbalanced dataset

In [ ]:
# Split the data into training and testing sets
X, X_test, y, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
minority_class = True
majority_class = False

# Count the number of instances in the minority class
minority_count = np.sum(y == minority_class)

# Count the number of instances in the majority class
majority_count = np.sum(y != minority_class)

# Calculate the oversampling ratio
oversampling_ratio = int(majority_count / minority_count)

# Indices of instances in the minority class
minority_indices = np.where(y == minority_class)[0]

# Oversample the minority class
oversampled_X = X[minority_indices]
oversampled_y = y[minority_indices]

for _ in range(oversampling_ratio - 1):
    oversampled_X = np.vstack((oversampled_X, X[minority_indices]))
    oversampled_y = np.hstack((oversampled_y, y[minority_indices]))

# Combine oversampled data with the majority class
X_resampled = np.vstack((X[y != minority_class], oversampled_X))
y_resampled = np.hstack((y[y != minority_class], oversampled_y))

# Now X_resampled and y_resampled contain the oversampled data

xgboost

In [ ]:
import xgboost as xgb
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


X_train, y_train = X_resampled, y_resampled

# Create a DMatrix for XGBoost (an internal data structure for efficient data handling)
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Set hyperparameters for the XGBoost model
params = {
    'max_depth': 3,
    'eta': 0.1,
    'objective': 'multi:softmax',  # Multiclass classification
    'num_class': 3,  # Number of classes in your target variable
}

num_round = 100  # Number of boosting rounds

# Train the XGBoost model
model = xgb.train(params, dtrain, num_round)

# Make predictions
y_pred = model.predict(dtest)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

PCA

In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# Apply PCA to reduce the data to 2D for visualization
pca = PCA(n_components=2)
X_reduced = pca.fit_transform(hashtag_clustering_.iloc[:,3:-1])

from sklearn.preprocessing import StandardScaler

# Create a StandardScaler object
scaler = StandardScaler()

# Fit and transform the data
#scaled_data = scaler.fit_transform(df.iloc[:,list(range(9,16))])
X_reduced = scaler.fit_transform(X_reduced)


# Visualize the results
plt.scatter(X_reduced[:, 0], X_reduced[:, 1], c=hashtag_clustering_['high_risk'], cmap='viridis')

# Add the legend
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.colorbar()
plt.title('Isolation Forest Anomaly Detection (PCA Visualization)')
plt.show()